In [10]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import os
load_dotenv()
import sys

#module_path = os.environ['MODULE_PATH']
#print(module_path)
#if module_path not in sys.path:
#    sys.path.append(module_path)

sys.path.append(os.path.join(os.environ['GIT_FOLDER']))
print(os.environ['GIT_FOLDER'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\Users\Ethan\SkyTruth\git\cerulean-cloud\


In [11]:
print(sys.path)

['C:\\Users\\Ethan', 'C:\\Users\\Ethan\\AppData\\Local\\anaconda3\\envs\\CeruleanWork\\python311.zip', 'C:\\Users\\Ethan\\AppData\\Local\\anaconda3\\envs\\CeruleanWork\\DLLs', 'C:\\Users\\Ethan\\AppData\\Local\\anaconda3\\envs\\CeruleanWork\\Lib', 'C:\\Users\\Ethan\\AppData\\Local\\anaconda3\\envs\\CeruleanWork', '', 'C:\\Users\\Ethan\\AppData\\Local\\anaconda3\\envs\\CeruleanWork\\Lib\\site-packages', 'C:\\Users\\Ethan\\AppData\\Local\\anaconda3\\envs\\CeruleanWork\\Lib\\site-packages\\win32', 'C:\\Users\\Ethan\\AppData\\Local\\anaconda3\\envs\\CeruleanWork\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\Ethan\\AppData\\Local\\anaconda3\\envs\\CeruleanWork\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\Ethan\\SkyTruth\\git\\cerulean-cloud\\cerulean_cloud\\models', 'C:\\Users\\Ethan\\SkyTruth\\git\\cerulean-cloud\\cerulean_cloud', 'C:\\Users\\Ethan\\SkyTruth\\git\\cerulean-cloud\\']


In [13]:
from cerulean_cloud.models import get_model
from cerulean_cloud.cloud_run_offset_tiles.schema import InferenceInput
from cerulean_cloud.cloud_run_orchestrator.clients import img_array_to_b64_image
import numpy as np

from cerulean_cloud.cloud_run_orchestrator.schema import OrchestratorInput
from cerulean_cloud.tiling import TMS, offset_bounds_from_base_tiles
from cerulean_cloud.titiler_client import TitilerClient
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'cerulean_cloud.models'

In [ ]:
async def get_titiler_client_and_offset_tiles(sentinel_scene, offset=.33):
    payload = OrchestratorInput(**sentinel_scene)
    TitilerClient_url = os.getenv('TITILER_URL')
    titiler_client = TitilerClient(url=TitilerClient_url)
    scene_bounds = await titiler_client.get_bounds(payload.sceneid)
    tiler = TMS
    base_tiles = list(tiler.tiles(*scene_bounds, [payload.zoom], truncate=False))
    offset_tile_bounds = offset_bounds_from_base_tiles(base_tiles, offset_amount=offset)
    return titiler_client, offset_tile_bounds

In [ ]:
dbs_dict = {
    "type": "FASTAIUNET",
    "file_path": "",#"experiments/2024_03_06_18_14_31_7cls_rn101_pr256_z9_fastai_baseline_noamb/tracing_cpu_model.pt",
    "layers": ["VV"],
    "cls_map": {
        0: "BACKGROUND",
        1: "INFRA",
        2: "NATURAL",
        3: "COIN_VESSEL",
        4: "REC_VESSEL",
        5: "OLD_VESSEL",
        6: "BACKGROUND"  # HITL AMBIGUOUS, should never be output by inference_idx
    },  # inference_idx maps to class table
    "name": "ResNet101 Baseline Noamb",
    "tile_width_m": 40844,  # Used to calculate zoom
    "tile_width_px": 256,  # Used to calculate scale
    "epochs": 80,
    "thresholds": {
        "pixel_nms_thresh": 0.4,
        "bbox_score_thresh": 0.001,
        "poly_score_thresh": 0.5,
        "pixel_score_thresh": 0.9995,
        "groundtruth_dice_thresh": 0.0
    },
    "backbone_size": 101
    # "pixel_f1": 0.0,  # TODO CALCULATE
    # "instance_f1": 0.0  # TODO CALCULATE
}

scene_id = "S1A_IW_GRDH_1SDV_20240204T184243_20240204T184308_052413_0656A2_1B88"
test_scene = {"sceneid": scene_id , "zoom":9, "scale":2}
offset2_client , offset_tiles2 =  await get_titiler_client_and_offset_tiles(test_scene,offset=.66)

example_tile = offset_tiles2[37]

vv = (await offset2_client.get_offset_tile(scene_id, *example_tile,height=512,width=512)).transpose(2,0,1)[0]

encoded = img_array_to_b64_image(np.array([vv]))
inf_stack = [InferenceInput(image=encoded, bounds=example_tile)]

model = get_model(dbs_dict,model_path_local=model_path)

preds = model.predict(inf_stack)

print(preds)

In [ ]:
plt.imshow(vv, cmap="grey")